In [7]:
import numpy as np
import pandas as pd
import os
import sys

# Add the directory containing the VotingModel to the Python path
sys.path.append(os.path.abspath('/Users/idrees/Code/govxs/'))
from util.voting_rules import mean_aggregation, median_aggregation, quadratic_aggregation

from model.VotingModel import VotingModel

def simulate_bribery_mean(model, target_project, desired_increase):
    num_voters, num_projects = model.voting_matrix.shape
    new_voting_matrix = model.voting_matrix.copy()
    original_voting_matrix=new_voting_matrix

    # Calculate the original allocation
    original_allocation = model.mean_aggregation()
    original_funds = original_allocation[target_project]

    # Calculate the target allocation
    target_funds = original_funds + desired_increase
    total_required_votes = target_funds * num_voters

    # Calculate the current votes for the target project
    current_votes = np.sum(new_voting_matrix[:, target_project])
    votes_needed = total_required_votes - current_votes

    bribery_cost = votes_needed
    
    return bribery_cost

def simulate_bribery_quadratic(model, target_project, desired_increase):
    num_voters, num_projects = model.voting_matrix.shape
    new_voting_matrix = model.voting_matrix.copy()

    # Calculate the original allocation
    original_allocation = model.quadratic_aggregation()
    original_funds = original_allocation[target_project]

    # Calculate the target allocation
    target_funds = original_funds + desired_increase
    total_required_votes = (target_funds * np.sum(original_allocation)) ** 0.5

    # Calculate the current votes for the target project
    current_votes = np.sum(new_voting_matrix[:, target_project] ** 2)
    votes_needed = total_required_votes - np.sqrt(current_votes)
    votes_needed = votes_needed ** 2

    bribery_cost = votes_needed
    
    
    return bribery_cost

def simulate_bribery_median(model, target_project, desired_increase):
    num_voters, num_projects = model.voting_matrix.shape
    new_voting_matrix = model.voting_matrix.copy()
    original_voting_matrix=new_voting_matrix

    # Calculate the original allocation
    original_allocation = model.median_aggregation()
    original_funds = original_allocation[target_project]

    # Calculate the target allocation
    target_funds = original_funds + desired_increase

    # Calculate the current median vote for the target project
    votes = new_voting_matrix[:, target_project]
    current_median_vote = np.median(votes)

    # Determine the new median vote required
    total_required_votes = target_funds * model.num_voters / model.total_op_tokens
    votes_needed = total_required_votes - current_median_vote
    

    bribery_cost = desired_increase

   
    
    return bribery_cost

def evaluate_bribery_impact(model, target_project, desired_increase):
    # Original allocations without bribery
    original_allocations = {}
    for method in ["mean", "median", "quadratic"]:
        original_allocations[method] = model.allocate_funds(method)[target_project]

    # Bribery impact for each aggregation method
    bribery_results = []
    for method in ["mean", "median", "quadratic"]:
        if method == "mean":
            bribery_cost= simulate_bribery_mean(
                model, target_project, desired_increase
            )
        elif method == "median":
            bribery_cost = simulate_bribery_median(
                model, target_project, desired_increase
            )
        elif method == "quadratic":
            bribery_cost = simulate_bribery_quadratic(
                model, target_project, desired_increase
            )
    
        bribery_results.append({
            "method": method,
            "bribery_cost": bribery_cost,
            
                    })

    return pd.DataFrame(bribery_results)
# Initialize the model
model = VotingModel(num_voters=3, num_projects=3, total_op_tokens=30e6)
model.step()

# Set parameters for bribery evaluation
target_project = 1
desired_increase = 10e6  # Desired increase of 5M OP tokens

# Evaluate the impact of bribery
bribery_impact_df = evaluate_bribery_impact(model, target_project, desired_increase)
bribery_impact_df.head()

/Users/idrees/Code/govxs/agents/VoterAgent.py:8: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)


,method,bribery_cost
0,mean,3.000000e+07
1,median,1.000000e+07
2,quadratic,3.449597e+13
